In [2]:
!pip install torch
!git clone https://github.com/neubig/nn4nlp-code.git

fatal: destination path 'nn4nlp-code' already exists and is not an empty directory.


In [0]:
from collections import defaultdict
import time
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

In [0]:
# Functions to read in the corpus
w2i = defaultdict(lambda: len(w2i))
t2i = defaultdict(lambda: len(t2i))
UNK = w2i["<unk>"]


def read_dataset(filename):
    with open(filename, "r") as f:
        for line in f:
            tag, words = line.lower().strip().split(" ||| ")
            yield ([w2i[x] for x in words.split(" ")], t2i[tag])


# Read in the data
train = list(read_dataset("nn4nlp-code/data/classes/train.txt"))
w2i = defaultdict(lambda: UNK, w2i)
dev = list(read_dataset("nn4nlp-code/data/classes/test.txt"))
nwords = len(w2i)
ntags = len(t2i)

In [0]:
def get_batch(source, i):
  seq_len = batch_size
  data = source[i*batch_size:min((i+1)*batch_size, len(source))]
  
  inputs, targets = zip(*data)
  
  max_len = max([len(x) for x in inputs])
  inputs = torch.stack([F.pad(torch.LongTensor(e), (max_len - len(e), 0)) for e in inputs], 0)
  targets = torch.LongTensor(targets)
  return inputs.cuda(), targets.cuda()

In [0]:
class SentimentModel(nn.Module):
  def __init__(self, vocab_size, embedding_dim, hidden_dim, num_class):
    super(SentimentModel, self).__init__()
    self.hidden_dim = hidden_dim
    
    self.embeddings = nn.Embedding(vocab_size, embedding_dim)
    self.lstm = nn.LSTM(embedding_dim, hidden_dim, bidirectional=True)
    self.linear = nn.Linear(hidden_dim*2, num_class)
    
  def init_hidden(self, batch_size):
    if torch.cuda.is_available():
      return (torch.zeros(2, batch_size, self.hidden_dim).cuda(), torch.zeros(2, batch_size, self.hidden_dim).cuda())
    else:
      return (torch.zeros(2, batch_size, self.hidden_dim), torch.zeros(2, batch_size, self.hidden_dim))
    
  def forward(self, sent):
    batch_size, seq_len = sent.shape
    hidden = self.init_hidden(batch_size)
    x = self.embeddings(sent).transpose(0,1)
    lstm_out, hidden = self.lstm(x, hidden)
    return self.linear(lstm_out[-1])

In [7]:
torch.cuda.is_available()

True

In [0]:
EMBED_DIM = 64
HIDDEN_DIM = 128
batch_size = 64

model = SentimentModel(nwords, EMBED_DIM, HIDDEN_DIM, ntags).cuda()

In [0]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [0]:
def get_accuracy(truth, pred):
  assert len(truth) == len(pred)
  right = 0
  for i in range(len(truth)):
    if truth[i].cpu().numpy() == pred[i]:
      right += 1.0
  return right / len(truth)

def evaluate(model, loss_function):
  model.eval()
  avg_loss = 0.0
  truth_res = []
  pred_res = []
  for ix in range(len(dev) // batch_size):
    sent, label = get_batch(dev, ix)
#     label.data.sub_(1)
    truth_res += list(label.data)
    pred = model(sent)
    pred_label = pred.data.max(1)[1].cpu().long().numpy()
    pred_res += [x for x in pred_label]
    loss = loss_function(pred, label)
    avg_loss += loss.data[0]
  avg_loss /= len(dev)
  acc = get_accuracy(truth_res, pred_res)
  print('eval: loss %.4f acc %.2f' % (avg_loss, acc*100))
  return acc

In [23]:
for epoch_i in range(10):
  model.train()
  total_loss = 0.
  for ix in range(len(train) // batch_size):
    model.zero_grad()

    inputs, targets = get_batch(train, ix)
    logit = model(inputs)
    loss = criterion(logit, targets)
    loss.backward()

    optimizer.step()
    total_loss += loss.item()

  print("Epoch: {} | train loss/sent: {}".format(epoch_i, total_loss / batch_size / (len(train) // batch_size)))
  
  evaluate(model, criterion)

Epoch: 0 | train loss/sent: 0.018923526839878326


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


eval: loss 0.0281 acc 29.00
Epoch: 1 | train loss/sent: 0.017219470281686102
eval: loss 0.0288 acc 29.69
Epoch: 2 | train loss/sent: 0.015084911536957537
eval: loss 0.0302 acc 30.51
Epoch: 3 | train loss/sent: 0.01301752532923356
eval: loss 0.0317 acc 30.61
Epoch: 4 | train loss/sent: 0.011400441989246616
eval: loss 0.0331 acc 30.10
Epoch: 5 | train loss/sent: 0.00981439536269334
eval: loss 0.0349 acc 30.01
Epoch: 6 | train loss/sent: 0.008510743148792955
eval: loss 0.0368 acc 29.92
Epoch: 7 | train loss/sent: 0.007502611085800524
eval: loss 0.0388 acc 28.54
Epoch: 8 | train loss/sent: 0.006587495701737646
eval: loss 0.0400 acc 29.27
Epoch: 9 | train loss/sent: 0.005823535296743862
eval: loss 0.0408 acc 29.69
